<a href="https://colab.research.google.com/github/davidlealo/vocacional-test/blob/main/Azure_Video_Indexer_to_jsonl_converter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Explicación del Código: Conversor de Azure Video Indexer a .jsonl

Este documento explica el código Python diseñado para convertir archivos exportados desde Azure Video Indexer (en formato ZIP) a un archivo `.jsonl` (JSON Lines). El código está optimizado para ejecutarse en Google Colab y procesa tres tipos de artefactos: transcripciones, OCR (reconocimiento óptico de caracteres) y etiquetas visuales. A continuación, se detalla cada sección del código, su propósito y funcionamiento.

---

## Resumen General
El script realiza los siguientes pasos:
1. Permite al usuario subir un archivo ZIP exportado desde Azure Video Indexer.
2. Extrae el contenido del ZIP a un directorio temporal.
3. Solicita metadatos contextuales (como nombre de la clase, fecha, curso y notas).
4. Procesa los archivos JSON de transcripción, OCR y etiquetas visuales, extrayendo información relevante.
5. Genera un archivo `.jsonl` con los datos procesados, incluyendo los metadatos proporcionados.
6. Descarga el archivo `.jsonl` y elimina los archivos temporales.

El código incluye manejo de errores robusto, mensajes claros y codificación UTF-8 para soportar caracteres especiales. Está diseñado para ser tolerante a fallos, como la ausencia de ciertos archivos o problemas en la lectura de JSON.

---

## Estructura del Código

### 1. Importación de Bibliotecas
```python
import zipfile
import os
import json
import shutil
from google.colab import files
from datetime import datetime
```
- **Propósito**: Importa las bibliotecas necesarias para:
  - Manejo de archivos ZIP (`zipfile`).
  - Operaciones con el sistema de archivos (`os`, `shutil`).
  - Procesamiento de JSON (`json`).
  - Interfaz de carga/descarga en Google Colab (`files`).
  - Validación de fechas (`datetime`).

---

### 2. Subida del Archivo ZIP
```python
print("Sube el archivo ZIP exportado desde Azure Video Indexer")
try:
    uploaded = files.upload()
    zip_path = next(iter(uploaded))
except Exception as e:
    print(f"Error al subir el archivo ZIP: {e}")
    raise
```
- **Propósito**: Solicita al usuario que suba un archivo ZIP desde Google Colab.
- **Funcionamiento**:
  - Usa `files.upload()` para abrir una interfaz de carga.
  - Extrae la ruta del archivo ZIP subido.
  - Captura cualquier error durante la subida (por ejemplo, si no se selecciona ningún archivo) y detiene la ejecución con un mensaje claro.

---

### 3. Extracción del Contenido del ZIP
```python
extract_dir = "azure_indexer_artifacts"
try:
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)
    print(f"Archivos extraídos correctamente en: {extract_dir}")
except zipfile.BadZipFile:
    print("Error: El archivo ZIP está dañado o no es válido")
    raise
except Exception as e:
    print(f"Error al extraer el archivo ZIP: {e}")
    raise
```
- **Propósito**: Extrae los archivos del ZIP a un directorio temporal (`azure_indexer_artifacts`).
- **Funcionamiento**:
  - Usa `zipfile.ZipFile` para abrir y extraer el contenido del ZIP.
  - Maneja errores específicos (ZIP dañado) y genéricos, mostrando mensajes descriptivos.
  - Imprime la confirmación de la extracción exitosa.

---

### 4. Solicitud de Metadatos
```python
print("\nIngresa información contextual para incluir como metadatos comunes")
clase = input("Nombre de la clase o sesión: ").strip()
fecha = input("Fecha (YYYY-MM-DD): ").strip()
curso = input("Curso o unidad (opcional): ").strip()
notas = input("Notas adicionales (opcional): ").strip()

try:
    datetime.strptime(fecha, "%Y-%m-%d")
except ValueError:
    print("Error: La fecha debe estar en formato YYYY-MM-DD")
    raise
```
- **Propósito**: Recolecta metadatos contextuales que se incluirán en cada entrada del archivo `.jsonl`.
- **Funcionamiento**:
  - Solicita al usuario el nombre de la clase, fecha, curso (opcional) y notas (opcional).
  - Usa `strip()` para eliminar espacios en blanco no deseados.
  - Valida que la fecha tenga el formato `YYYY-MM-DD` usando `datetime.strptime`. Si el formato es incorrecto, lanza un error.

---

### 5. Funciones para Procesar Artefactos
El código define tres funciones para procesar los archivos JSON generados por Azure Video Indexer: transcripción, OCR y etiquetas visuales. Cada función tiene una estructura similar.

#### a. `extract_transcript`
```python
def extract_transcript(path):
    try:
        with open(path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        entries = []
        for segment in data.get("recognizedPhrases", []):
            text = segment.get("display", "")
            if text:
                entries.append({
                    "text": text,
                    "source": "transcript",
                    "metadata": {
                        "clase": clase,
                        "fecha": fecha,
                        "curso": curso,
                        "notas": notas,
                        "tipo": "transcript"
                    }
                })
        return entries
    except Exception as e:
        print(f"Error al procesar el archivo de transcripción: {e}")
        return []
```
- **Propósito**: Procesa el archivo de transcripción (`transcript.speechservices.json`).
- **Funcionamiento**:
  - Lee el archivo JSON con codificación UTF-8.
  - Extrae los segmentos de texto de la clave `"recognizedPhrases"`.
  - Crea una entrada por cada segmento de texto, incluyendo el texto, la fuente (`transcript`) y los metadatos.
  - Si hay un error (por ejemplo, archivo corrupto o formato inesperado), muestra un mensaje y devuelve una lista vacía.

#### b. `extract_ocr`
```python
def extract_ocr(path):
    try:
        with open(path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        entries = []
        for region in data.get("regions", []):
            for line in region.get("lines", []):
                text = line.get("text", "")
                if text:
                    entries.append({
                        "text": text,
                        "source": "ocr",
                        "metadata": {
                            "clase": clase,
                            "fecha": fecha,
                            "curso": curso,
                            "notas": notas,
                            "tipo": "ocr"
                        }
                    })
        return entries
    except Exception as e:
        print(f"Error al procesar el archivo OCR: {e}")
        return []
```
- **Propósito**: Procesa el archivo OCR (`ocr.json`).
- **Funcionamiento**:
  - Lee el archivo JSON y extrae el texto de las claves `"regions"` y `"lines"`.
  - Crea una entrada por cada línea de texto detectada, con la fuente `ocr` y los metadatos.
  - Maneja errores de manera similar a `extract_transcript`.

#### c. `extract_labels`
```python
def extract_labels(path):
    try:
        with open(path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        entries = []
        for label in data.get("labels", []):
            name = label.get("name", "")
            if name:
                entries.append({
                    "text": name,
                    "source": "visual_label",
                    "metadata": {
                        "clase": clase,
                        "fecha": fecha,
                        "curso": curso,
                        "notas": notas,
                        "tipo": "label"
                    }
                })
        return entries
    except Exception as e:
        print(f"Error al procesar el archivo de etiquetas: {e}")
        return []
```
- **Propósito**: Procesa el archivo de etiquetas visuales (`labels.computervision.json`).
- **Funcionamiento**:
  - Extrae los nombres de etiquetas de la clave `"labels"`.
  - Crea una entrada por cada etiqueta, con la fuente `visual_label` y los metadatos.
  - Maneja errores devolviendo una lista vacía.

---

### 6. Procesamiento de Archivos
```python
jsonl_entries = []
paths = os.listdir(extract_dir)

if "transcript.speechservices.json" in paths:
    jsonl_entries.extend(extract_transcript(os.path.join(extract_dir, "transcript.speechservices.json")))
else:
    print("Advertencia: No se encontró el archivo de transcripción (transcript.speechservices.json)")

if "ocr.json" in paths:
    jsonl_entries.extend(extract_ocr(os.path.join(extract_dir, "ocr.json")))
else:
    print("Advertencia: No se encontró el archivo OCR (ocr.json)")

if "labels.computervision.json" in paths:
    jsonl_entries.extend(extract_labels(os.path.join(extract_dir, "labels.computervision.json")))
else:
    print("Advertencia: No se encontró el archivo de etiquetas (labels.computervision.json)")
```
- **Propósito**: Busca y procesa los tres archivos esperados en el directorio extraído.
- **Funcionamiento**:
  - Revisa si cada archivo (`transcript.speechservices.json`, `ocr.json`, `labels.computervision.json`) está presente.
  - Si un archivo existe, llama a la función correspondiente (`extract_transcript`, `extract_ocr`, `extract_labels`) y agrega las entradas a `jsonl_entries`.
  - Si un archivo no está presente, muestra una advertencia pero continúa con los demás.

---

### 7. Exportación del Archivo `.jsonl`
```python
if jsonl_entries:
    output_file = f"video_indexed_{clase.replace(' ', '_')}.jsonl"
    try:
        with open(output_file, "w", encoding='utf-8') as f:
            for entry in jsonl_entries:
                f.write(json.dumps(entry, ensure_ascii=False) + "\n")
        print(f"Archivo exportado correctamente: {output_file}")
        files.download(output_file)
    except Exception as e:
        print(f"Error al exportar el archivo .jsonl: {e}")
        raise
else:
    print("Error: No se encontraron datos válidos para exportar")
```
- **Propósito**: Genera y descarga el archivo `.jsonl` con los datos procesados.
- **Funcionamiento**:
  - Verifica si hay entradas válidas en `jsonl_entries`.
  - Crea un archivo `.jsonl` con el nombre basado en `clase` (reemplazando espacios por guiones bajos).
  - Escribe cada entrada como una línea JSON, usando `ensure_ascii=False` para soportar caracteres no ASCII.
  - Descarga el archivo usando `files.download`.
  - Si no hay entradas válidas, muestra un error. Maneja errores durante la escritura.

---

### 8. Limpieza
```python
try:
    shutil.rmtree(extract_dir)
    os.remove(zip_path)
    print("Archivos temporales eliminados correctamente")
except Exception as e:
    print(f"Advertencia: Error al limpiar archivos temporales: {e}")
```
- **Propósito**: Elimina el directorio temporal y el archivo ZIP original.
- **Funcionamiento**:
  - Usa `shutil.rmtree` para eliminar el directorio `azure_indexer_artifacts`.
  - Usa `os.remove` para eliminar el archivo ZIP.
  - Maneja errores mostrando una advertencia, pero no detiene el programa.

---

## Características Clave
- **Tolerancia a fallos**: El código maneja errores en la subida, extracción, procesamiento y escritura, mostrando mensajes claros y continuando cuando es posible.
- **Soporte para caracteres especiales**: Usa codificación UTF-8 en todas las operaciones de archivo.
- **Mensajes claros**: Los mensajes son descriptivos y sin emojis, con un tono profesional.
- **Validación de datos**: Incluye verificación del formato de fecha y manejo de archivos faltantes.
- **Compatibilidad con Google Colab**: Utiliza la API de Colab para subir y descargar archivos.

---

## Posibles Mejoras
- **Validación adicional**: Verificar que el ZIP contenga al menos uno de los archivos esperados antes de procesar.
- **Configuración flexible**: Permitir nombres de archivo personalizables o un directorio de salida configurable.
- **Progreso visual**: Añadir una barra de progreso (por ejemplo, con `tqdm`) para archivos grandes.
- **Logs detallados**: Guardar un registro de errores y advertencias en un archivo para depuración.

---

## Ejemplo de Uso
1. Ejecuta el código en Google Colab.
2. Sube un archivo ZIP exportado desde Azure Video Indexer.
3. Ingresa los metadatos solicitados (clase, fecha, curso, notas).
4. El script procesa los archivos, genera un `.jsonl` y lo descarga automáticamente.
5. Los archivos temporales se eliminan.

El archivo `.jsonl` resultante contiene entradas con el formato:
```json
{"text": "texto extraído", "source": "transcript|ocr|visual_label", "metadata": {"clase": "...", "fecha": "...", "curso": "...", "notas": "...", "tipo": "..."}}
```

Este formato es ideal para análisis posterior, como búsquedas o indexación en bases de datos.

In [1]:
# Azure Video Indexer to .jsonl converter
# Compatible con Google Colab

import zipfile
import os
import json
import shutil
from google.colab import files
from datetime import datetime

# --- Paso 1: Subir archivo ZIP ---
print("Sube el archivo ZIP exportado desde Azure Video Indexer")
try:
    uploaded = files.upload()
    zip_path = next(iter(uploaded))
except Exception as e:
    print(f"Error al subir el archivo ZIP: {e}")
    raise

extract_dir = "azure_indexer_artifacts"

# --- Paso 2: Extraer contenido del ZIP ---
try:
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)
    print(f"Archivos extraídos correctamente en: {extract_dir}")
except zipfile.BadZipFile:
    print("Error: El archivo ZIP está dañado o no es válido")
    raise
except Exception as e:
    print(f"Error al extraer el archivo ZIP: {e}")
    raise

# --- Paso 3: Pedir metadatos generales ---
print("\nIngresa información contextual para incluir como metadatos comunes")
clase = input("Nombre de la clase o sesión: ").strip()
fecha = input("Fecha (YYYY-MM-DD): ").strip()
curso = input("Curso o unidad (opcional): ").strip()
notas = input("Notas adicionales (opcional): ").strip()

# Validar formato de fecha
try:
    datetime.strptime(fecha, "%Y-%m-%d")
except ValueError:
    print("Error: La fecha debe estar en formato YYYY-MM-DD")
    raise

# --- Paso 4: Funciones para procesar artefactos ---
def extract_transcript(path):
    try:
        with open(path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        entries = []
        for segment in data.get("recognizedPhrases", []):
            text = segment.get("display", "")
            if text:
                entries.append({
                    "text": text,
                    "source": "transcript",
                    "metadata": {
                        "clase": clase,
                        "fecha": fecha,
                        "curso": curso,
                        "notas": notas,
                        "tipo": "transcript"
                    }
                })
        return entries
    except Exception as e:
        print(f"Error al procesar el archivo de transcripción: {e}")
        return []

def extract_ocr(path):
    try:
        with open(path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        entries = []
        for region in data.get("regions", []):
            for line in region.get("lines", []):
                text = line.get("text", "")
                if text:
                    entries.append({
                        "text": text,
                        "source": "ocr",
                        "metadata": {
                            "clase": clase,
                            "fecha": fecha,
                            "curso": curso,
                            "notas": notas,
                            "tipo": "ocr"
                        }
                    })
        return entries
    except Exception as e:
        print(f"Error al procesar el archivo OCR: {e}")
        return []

def extract_labels(path):
    try:
        with open(path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        entries = []
        for label in data.get("labels", []):
            name = label.get("name", "")
            if name:
                entries.append({
                    "text": name,
                    "source": "visual_label",
                    "metadata": {
                        "clase": clase,
                        "fecha": fecha,
                        "curso": curso,
                        "notas": notas,
                        "tipo": "label"
                    }
                })
        return entries
    except Exception as e:
        print(f"Error al procesar el archivo de etiquetas: {e}")
        return []

# --- Paso 5: Procesar los tres artefactos clave ---
jsonl_entries = []
paths = os.listdir(extract_dir)

if "transcript.speechservices.json" in paths:
    jsonl_entries.extend(extract_transcript(os.path.join(extract_dir, "transcript.speechservices.json")))
else:
    print("Advertencia: No se encontró el archivo de transcripción (transcript.speechservices.json)")

if "ocr.json" in paths:
    jsonl_entries.extend(extract_ocr(os.path.join(extract_dir, "ocr.json")))
else:
    print("Advertencia: No se encontró el archivo OCR (ocr.json)")

if "labels.computervision.json" in paths:
    jsonl_entries.extend(extract_labels(os.path.join(extract_dir, "labels.computervision.json")))
else:
    print("Advertencia: No se encontró el archivo de etiquetas (labels.computervision.json)")

# --- Paso 6: Exportar .jsonl ---
if jsonl_entries:
    output_file = f"video_indexed_{clase.replace(' ', '_')}.jsonl"
    try:
        with open(output_file, "w", encoding='utf-8') as f:
            for entry in jsonl_entries:
                f.write(json.dumps(entry, ensure_ascii=False) + "\n")
        print(f"Archivo exportado correctamente: {output_file}")
        files.download(output_file)
    except Exception as e:
        print(f"Error al exportar el archivo .jsonl: {e}")
        raise
else:
    print("Error: No se encontraron datos válidos para exportar")

# --- Paso 7: Limpieza ---
try:
    shutil.rmtree(extract_dir)
    os.remove(zip_path)
    print("Archivos temporales eliminados correctamente")
except Exception as e:
    print(f"Advertencia: Error al limpiar archivos temporales: {e}")

Sube el archivo ZIP exportado desde Azure Video Indexer


Saving live.zip to live.zip
Archivos extraídos correctamente en: azure_indexer_artifacts

Ingresa información contextual para incluir como metadatos comunes
Nombre de la clase o sesión: Live entre Valentina Gran y David Leal en el Instagram de Fundación Por Una Carrera el 23 de junio en https://www.instagram.com/porunacarrera/reel/DLTQtETxUUF/ 
Fecha (YYYY-MM-DD): 2025-06-23
Curso o unidad (opcional): 
Notas adicionales (opcional): 
Advertencia: No se encontró el archivo de transcripción (transcript.speechservices.json)
Advertencia: No se encontró el archivo OCR (ocr.json)
Advertencia: No se encontró el archivo de etiquetas (labels.computervision.json)
Error: No se encontraron datos válidos para exportar
Archivos temporales eliminados correctamente


In [3]:
# Azure Video Indexer to .jsonl converter
# Compatible con Google Colab

import zipfile
import os
import json
import shutil
from google.colab import files
from datetime import datetime

# --- Paso 1: Subir archivo ZIP ---
print("Sube el archivo ZIP exportado desde Azure Video Indexer")
try:
    uploaded = files.upload()
    zip_path = next(iter(uploaded))
except Exception as e:
    print(f"Error al subir el archivo ZIP: {e}")
    raise

extract_dir = "azure_indexer_artifacts"

# --- Paso 2: Extraer contenido del ZIP ---
try:
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)
    print(f"Archivos extraídos correctamente en: {extract_dir}")
except zipfile.BadZipFile:
    print("Error: El archivo ZIP está dañado o no es válido")
    raise
except Exception as e:
    print(f"Error al extraer el archivo ZIP: {e}")
    raise

# Buscar subdirectorios si los archivos no están en la raíz (común en exportaciones de Azure)
root_files = os.listdir(extract_dir)
if len(root_files) == 1 and os.path.isdir(os.path.join(extract_dir, root_files[0])):
    extract_dir = os.path.join(extract_dir, root_files[0])
    print(f"Archivos encontrados en subdirectorio: {extract_dir}")

# --- Paso 3: Pedir metadatos generales ---
print("\nIngresa información contextual para incluir como metadatos comunes")
clase = input("Nombre de la clase o sesión: ").strip()
fecha = input("Fecha (YYYY-MM-DD): ").strip()
curso = input("Curso o unidad (opcional): ").strip()
notas = input("Notas adicionales (opcional): ").strip()

# Validar formato de fecha
try:
    datetime.strptime(fecha, "%Y-%m-%d")
except ValueError:
    print("Error: La fecha debe estar en formato YYYY-MM-DD")
    raise

# --- Paso 4: Funciones para procesar artefactos ---
def extract_transcript(path):
    try:
        with open(path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        entries = []
        for segment in data.get("recognizedPhrases", []):
            text = segment.get("display", "")
            if text:
                entries.append({
                    "text": text,
                    "source": "transcript",
                    "metadata": {
                        "clase": clase,
                        "fecha": fecha,
                        "curso": curso,
                        "notas": notas,
                        "tipo": "transcript"
                    }
                })
        return entries
    except Exception as e:
        print(f"Error al procesar el archivo de transcripción: {e}")
        return []

def extract_ocr(path):
    try:
        with open(path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        entries = []
        for region in data.get("regions", []):
            for line in region.get("lines", []):
                text = line.get("text", "")
                if text:
                    entries.append({
                        "text": text,
                        "source": "ocr",
                        "metadata": {
                            "clase": clase,
                            "fecha": fecha,
                            "curso": curso,
                            "notas": notas,
                            "tipo": "ocr"
                        }
                    })
        return entries
    except Exception as e:
        print(f"Error al procesar el archivo OCR: {e}")
        return []

def extract_labels(path):
    try:
        with open(path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        entries = []
        for label in data.get("labels", []):
            name = label.get("name", "")
            if name:
                entries.append({
                    "text": name,
                    "source": "visual_label",
                    "metadata": {
                        "clase": clase,
                        "fecha": fecha,
                        "curso": curso,
                        "notas": notas,
                        "tipo": "label"
                    }
                })
        return entries
    except Exception as e:
        print(f"Error al procesar el archivo de etiquetas: {e}")
        return []

# --- Paso 5: Procesar los tres artefactos clave ---
jsonl_entries = []
paths = os.listdir(extract_dir)

if "transcript.speechservices.json" in paths:
    jsonl_entries.extend(extract_transcript(os.path.join(extract_dir, "transcript.speechservices.json")))
else:
    print("Advertencia: No se encontró el archivo de transcripción (transcript.speechservices.json)")

if "ocr.json" in paths:
    jsonl_entries.extend(extract_ocr(os.path.join(extract_dir, "ocr.json")))
else:
    print("Advertencia: No se encontró el archivo OCR (ocr.json)")

if "labels.computervision.json" in paths:
    jsonl_entries.extend(extract_labels(os.path.join(extract_dir, "labels.computervision.json")))
else:
    print("Advertencia: No se encontró el archivo de etiquetas (labels.computervision.json)")

# --- Paso 6: Exportar .jsonl ---
if jsonl_entries:
    output_file = f"video_indexed_{clase.replace(' ', '_')}.jsonl"
    try:
        with open(output_file, "w", encoding='utf-8') as f:
            for entry in jsonl_entries:
                f.write(json.dumps(entry, ensure_ascii=False) + "\n")
        print(f"Archivo exportado correctamente: {output_file}")
        files.download(output_file)
    except Exception as e:
        print(f"Error al exportar el archivo .jsonl: {e}")
        raise
else:
    print("Error: No se encontraron datos válidos para exportar")

# --- Paso 7: Limpieza ---
try:
    shutil.rmtree("azure_indexer_artifacts")  # Limpiar el directorio raíz original
    os.remove(zip_path)
    print("Archivos temporales eliminados correctamente")
except Exception as e:
    print(f"Advertencia: Error al limpiar archivos temporales: {e}")

Sube el archivo ZIP exportado desde Azure Video Indexer


Saving live.zip to live.zip
Archivos extraídos correctamente en: azure_indexer_artifacts

Ingresa información contextual para incluir como metadatos comunes
Nombre de la clase o sesión: Live entre Valentina Gran y David Leal en el Instagram de Fundación Por Una Carrera el 23 de junio en https://www.instagram.com/porunacarrera/reel/DLTQtETxUUF/
Fecha (YYYY-MM-DD): 2025-06-23
Curso o unidad (opcional): Uso de Proyectate info
Notas adicionales (opcional): Este fue un live de demostracion de la aplicacion creada en conjunto 
Advertencia: No se encontró el archivo de transcripción (transcript.speechservices.json)
Advertencia: No se encontró el archivo OCR (ocr.json)
Advertencia: No se encontró el archivo de etiquetas (labels.computervision.json)
Error: No se encontraron datos válidos para exportar
Archivos temporales eliminados correctamente


In [4]:
# Azure Video Indexer to .jsonl converter
# Compatible con Google Colab

import zipfile
import os
import json
import shutil
from google.colab import files
from datetime import datetime

# --- Paso 1: Subir archivo ZIP ---
print("Sube el archivo ZIP exportado desde Azure Video Indexer")
try:
    uploaded = files.upload()
    zip_path = next(iter(uploaded))
except Exception as e:
    print(f"Error al subir el archivo ZIP: {e}")
    raise

extract_dir = "azure_indexer_artifacts"

# --- Paso 2: Extraer contenido del ZIP ---
try:
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)
    print(f"Archivos extraídos correctamente en: {extract_dir}")
except zipfile.BadZipFile:
    print("Error: El archivo ZIP está dañado o no es válido")
    raise
except Exception as e:
    print(f"Error al extraer el archivo ZIP: {e}")
    raise

# --- Depuración: Mostrar estructura del directorio extraído ---
print("\nEstructura del directorio extraído:")
for root, dirs, files in os.walk(extract_dir):
    for file in files:
        print(f" - {os.path.join(root, file)}")

# --- Paso 3: Pedir metadatos generales ---
print("\nIngresa información contextual para incluir como metadatos comunes")
clase = input("Nombre de la clase o sesión: ").strip()
fecha = input("Fecha (YYYY-MM-DD): ").strip()
curso = input("Curso o unidad (opcional): ").strip()
notas = input("Notas adicionales (opcional): ").strip()

# Validar formato de fecha
try:
    datetime.strptime(fecha, "%Y-%m-%d")
except ValueError:
    print("Error: La fecha debe estar en formato YYYY-MM-DD")
    raise

# --- Paso 4: Funciones para procesar artefactos ---
def extract_transcript(path):
    try:
        with open(path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        entries = []
        for segment in data.get("CombinedRecognizedPhrases", []):
            text = segment.get("DisplayText", "")
            if text:
                entries.append({
                    "text": text,
                    "source": "transcript",
                    "metadata": {
                        "clase": clase,
                        "fecha": fecha,
                        "curso": curso,
                        "notas": notas,
                        "tipo": "transcript"
                    }
                })
        return entries
    except Exception as e:
        print(f"Error al procesar el archivo de transcripción: {e}")
        return []

def extract_ocr(path):
    try:
        with open(path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        entries = []
        for result in data.get("Results", []):
            text = result.get("Ocr", {}).get("content", "")
            if text:
                entries.append({
                    "text": text,
                    "source": "ocr",
                    "metadata": {
                        "clase": clase,
                        "fecha": fecha,
                        "curso": curso,
                        "notas": notas,
                        "tipo": "ocr"
                    }
                })
        return entries
    except Exception as e:
        print(f"Error al procesar el archivo OCR: {e}")
        return []

def extract_labels(path):
    try:
        with open(path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        entries = []
        for result in data.get("Results", []):
            for label in result.get("Label", []):
                name = label.get("name", "")
                if name:
                    entries.append({
                        "text": name,
                        "source": "visual_label",
                        "metadata": {
                            "clase": clase,
                            "fecha": fecha,
                            "curso": curso,
                            "notas": notas,
                            "tipo": "label"
                        }
                    })
        return entries
    except Exception as e:
        print(f"Error al procesar el archivo de etiquetas: {e}")
        return []

def extract_emotions(path):
    try:
        with open(path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        entries = []
        for emotion in data:
            text = emotion.get("Text", "")
            dominant_emotion = emotion.get("DominantEmotion", {}).get("Type", "")
            if text and dominant_emotion:
                entries.append({
                    "text": f"{text} [Dominant Emotion: {dominant_emotion}]",
                    "source": "emotion",
                    "metadata": {
                        "clase": clase,
                        "fecha": fecha,
                        "curso": curso,
                        "notas": notas,
                        "tipo": "emotion",
                        "dominant_emotion": dominant_emotion
                    }
                })
        return entries
    except Exception as e:
        print(f"Error al procesar el archivo de emociones: {e}")
        return []

def extract_detected_objects(path):
    try:
        with open(path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        entries = []
        for result in data.get("results", []):
            obj_type = result.get("type", "")
            for instance in result.get("instances", []):
                if instance.get("confidence", 0) > 0:
                    entries.append({
                        "text": obj_type,
                        "source": "detected_object",
                        "metadata": {
                            "clase": clase,
                            "fecha": fecha,
                            "curso": curso,
                            "notas": notas,
                            "tipo": "detected_object",
                            "confidence": instance.get("confidence", 0)
                        }
                    })
        return entries
    except Exception as e:
        print(f"Error al procesar el archivo de objetos detectados: {e}")
        return []

# --- Paso 5: Buscar y procesar los archivos clave ---
jsonl_entries = []

# Buscar archivos recursivamente
file_paths = {}
for root, _, files in os.walk(extract_dir):
    for file in files:
        file_paths[file] = os.path.join(root, file)

# Procesar archivos si se encuentran
if "transcript.speechservices.json" in file_paths:
    print(f"Procesando transcripción: {file_paths['transcript.speechservices.json']}")
    jsonl_entries.extend(extract_transcript(file_paths["transcript.speechservices.json"]))
else:
    print("Advertencia: No se encontró el archivo de transcripción (transcript.speechservices.json)")

if "ocr.json" in file_paths:
    print(f"Procesando OCR: {file_paths['ocr.json']}")
    jsonl_entries.extend(extract_ocr(file_paths["ocr.json"]))
else:
    print("Advertencia: No se encontró el archivo OCR (ocr.json)")

if "labels.computervision.json" in file_paths:
    print(f"Procesando etiquetas: {file_paths['labels.computervision.json']}")
    jsonl_entries.extend(extract_labels(file_paths["labels.computervision.json"]))
else:
    print("Advertencia: No se encontró el archivo de etiquetas (labels.computervision.json)")

if "emotions.json" in file_paths:
    print(f"Procesando emociones: {file_paths['emotions.json']}")
    jsonl_entries.extend(extract_emotions(file_paths["emotions.json"]))
else:
    print("Advertencia: No se encontró el archivo de emociones (emotions.json)")

if "detectedobjects.json" in file_paths:
    print(f"Procesando objetos detectados: {file_paths['detectedobjects.json']}")
    jsonl_entries.extend(extract_detected_objects(file_paths["detectedobjects.json"]))
else:
    print("Advertencia: No se encontró el archivo de objetos detectados (detectedobjects.json)")

# --- Paso 6: Exportar .jsonl ---
if jsonl_entries:
    output_file = f"video_indexed_{clase.replace(' ', '_')}.jsonl"
    try:
        with open(output_file, "w", encoding='utf-8') as f:
            for entry in jsonl_entries:
                f.write(json.dumps(entry, ensure_ascii=False) + "\n")
        print(f"Archivo exportado correctamente: {output_file}")
        files.download(output_file)
    except Exception as e:
        print(f"Error al exportar el archivo .jsonl: {e}")
        raise
else:
    print("Error: No se encontraron datos válidos para exportar")

# --- Paso 7: Limpieza ---
try:
    shutil.rmtree("azure_indexer_artifacts")  # Limpiar el directorio raíz original
    os.remove(zip_path)
    print("Archivos temporales eliminados correctamente")
except Exception as e:
    print(f"Advertencia: Error al limpiar archivos temporales: {e}")

Sube el archivo ZIP exportado desde Azure Video Indexer


Saving live.zip to live.zip
Archivos extraídos correctamente en: azure_indexer_artifacts

Estructura del directorio extraído:
 - azure_indexer_artifacts/__MACOSX/._live
 - azure_indexer_artifacts/__MACOSX/live/._detectedobjects.json
 - azure_indexer_artifacts/__MACOSX/live/._labels.computervision.json
 - azure_indexer_artifacts/__MACOSX/live/._transcript.speechservices.json
 - azure_indexer_artifacts/__MACOSX/live/._Textual_Content_Moderation.json
 - azure_indexer_artifacts/__MACOSX/live/._ocr.json
 - azure_indexer_artifacts/__MACOSX/live/.__KeyFrameThumbnail.zip
 - azure_indexer_artifacts/__MACOSX/live/._emotions.json
 - azure_indexer_artifacts/__MACOSX/live/._contentmoderation.json
 - azure_indexer_artifacts/live/emotions.json
 - azure_indexer_artifacts/live/Textual_Content_Moderation.json
 - azure_indexer_artifacts/live/_KeyFrameThumbnail.zip
 - azure_indexer_artifacts/live/transcript.speechservices.json
 - azure_indexer_artifacts/live/contentmoderation.json
 - azure_indexer_artifac

FileNotFoundError: [Errno 2] No such file or directory: 'video_indexed_Live_entre_Valentina_Gran_y_David_Leal_en_el_Instagram_de_Fundación_Por_Una_Carrera_el_23_de_junio_en_https://www.instagram.com/porunacarrera/reel/DLTQtETxUUF/.jsonl'

In [5]:
# Azure Video Indexer to .jsonl converter
# Compatible con Google Colab

import zipfile
import os
import json
import shutil
from google.colab import files
from datetime import datetime
import re

# --- Paso 1: Subir archivo ZIP ---
print("Sube el archivo ZIP exportado desde Azure Video Indexer")
try:
    uploaded = files.upload()
    zip_path = next(iter(uploaded))
except Exception as e:
    print(f"Error al subir el archivo ZIP: {e}")
    raise

extract_dir = "azure_indexer_artifacts"

# --- Paso 2: Extraer contenido del ZIP ---
try:
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)
    print(f"Archivos extraídos correctamente en: {extract_dir}")
except zipfile.BadZipFile:
    print("Error: El archivo ZIP está dañado o no es válido")
    raise
except Exception as e:
    print(f"Error al extraer el archivo ZIP: {e}")
    raise

# --- Depuración: Mostrar estructura del directorio extraído ---
print("\nEstructura del directorio extraído:")
for root, dirs, files in os.walk(extract_dir):
    for file in files:
        print(f" - {os.path.join(root, file)}")

# --- Paso 3: Pedir metadatos generales ---
print("\nIngresa información contextual para incluir como metadatos comunes")
clase = input("Nombre de la clase o sesión: ").strip()
fecha = input("Fecha (YYYY-MM-DD): ").strip()
curso = input("Curso o unidad (opcional): ").strip()
notas = input("Notas adicionales (opcional): ").strip()

# Validar formato de fecha
try:
    datetime.strptime(fecha, "%Y-%m-%d")
except ValueError:
    print("Error: La fecha debe estar en formato YYYY-MM-DD")
    raise

# --- Paso 4: Funciones para procesar artefactos ---
def extract_transcript(path):
    try:
        with open(path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        entries = []
        for phrase in data.get("RecognizedPhrases", []):
            nbest = phrase.get("NBest", [])
            if nbest:
                text = nbest[0].get("Display", "")
                if text:
                    entries.append({
                        "text": text,
                        "source": "transcript",
                        "metadata": {
                            "clase": clase,
                            "fecha": fecha,
                            "curso": curso,
                            "notas": notas,
                            "tipo": "transcript"
                        }
                    })
        return entries
    except Exception as e:
        print(f"Error al procesar el archivo de transcripción: {e}")
        return []

def extract_ocr(path):
    try:
        with open(path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        entries = []
        for result in data.get("Results", []):
            text = result.get("Ocr", {}).get("content", "")
            if text:
                entries.append({
                    "text": text,
                    "source": "ocr",
                    "metadata": {
                        "clase": clase,
                        "fecha": fecha,
                        "curso": curso,
                        "notas": notas,
                        "tipo": "ocr"
                    }
                })
        return entries
    except Exception as e:
        print(f"Error al procesar el archivo OCR: {e}")
        return []

def extract_labels(path):
    try:
        with open(path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        entries = []
        for result in data.get("Results", []):
            for label in result.get("Label", []):
                name = label.get("name", "")
                if name:
                    entries.append({
                        "text": name,
                        "source": "visual_label",
                        "metadata": {
                            "clase": clase,
                            "fecha": fecha,
                            "curso": curso,
                            "notas": notas,
                            "tipo": "label"
                        }
                    })
        return entries
    except Exception as e:
        print(f"Error al procesar el archivo de etiquetas: {e}")
        return []

def extract_emotions(path):
    try:
        with open(path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        entries = []
        for emotion in data:
            text = emotion.get("Text", "")
            dominant_emotion = emotion.get("DominantEmotion", {}).get("Type", "")
            if text and dominant_emotion:
                entries.append({
                    "text": f"{text} [Dominant Emotion: {dominant_emotion}]",
                    "source": "emotion",
                    "metadata": {
                        "clase": clase,
                        "fecha": fecha,
                        "curso": curso,
                        "notas": notas,
                        "tipo": "emotion",
                        "dominant_emotion": dominant_emotion
                    }
                })
        return entries
    except Exception as e:
        print(f"Error al procesar el archivo de emociones: {e}")
        return []

def extract_detected_objects(path):
    try:
        with open(path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        entries = []
        for result in data.get("results", []):
            obj_type = result.get("type", "")
            for instance in result.get("instances", []):
                if instance.get("confidence", 0) > 0:
                    entries.append({
                        "text": obj_type,
                        "source": "detected_object",
                        "metadata": {
                            "clase": clase,
                            "fecha": fecha,
                            "curso": curso,
                            "notas": notas,
                            "tipo": "detected_object",
                            "confidence": instance.get("confidence", 0)
                        }
                    })
        return entries
    except Exception as e:
        print(f"Error al procesar el archivo de objetos detectados: {e}")
        return []

# --- Paso 5: Buscar y procesar los archivos clave ---
jsonl_entries = []

# Buscar archivos recursivamente
file_paths = {}
for root, _, files in os.walk(extract_dir):
    for file in files:
        file_paths[file] = os.path.join(root, file)

# Procesar archivos si se encuentran
if "transcript.speechservices.json" in file_paths:
    print(f"Procesando transcripción: {file_paths['transcript.speechservices.json']}")
    jsonl_entries.extend(extract_transcript(file_paths["transcript.speechservices.json"]))
else:
    print("Advertencia: No se encontró el archivo de transcripción (transcript.speechservices.json)")

if "ocr.json" in file_paths:
    print(f"Procesando OCR: {file_paths['ocr.json']}")
    jsonl_entries.extend(extract_ocr(file_paths["ocr.json"]))
else:
    print("Advertencia: No se encontró el archivo OCR (ocr.json)")

if "labels.computervision.json" in file_paths:
    print(f"Procesando etiquetas: {file_paths['labels.computervision.json']}")
    jsonl_entries.extend(extract_labels(file_paths["labels.computervision.json"]))
else:
    print("Advertencia: No se encontró el archivo de etiquetas (labels.computervision.json)")

if "emotions.json" in file_paths:
    print(f"Procesando emociones: {file_paths['emotions.json']}")
    jsonl_entries.extend(extract_emotions(file_paths["emotions.json"]))
else:
    print("Advertencia: No se encontró el archivo de emociones (emotions.json)")

if "detectedobjects.json" in file_paths:
    print(f"Procesando objetos detectados: {file_paths['detectedobjects.json']}")
    jsonl_entries.extend(extract_detected_objects(file_paths["detectedobjects.json"]))
else:
    print("Advertencia: No se encontró el archivo de objetos detectados (detectedobjects.json)")

# --- Paso 6: Exportar .jsonl ---
if jsonl_entries:
    sanitized_clase = re.sub(r'[/:*?"<>|]', '_', clase)
    sanitized_clase = sanitized_clase.replace(' ', '_')[:100]  # Truncar a 100 chars
    output_file = f"video_indexed_{sanitized_clase}.jsonl"
    try:
        with open(output_file, "w", encoding='utf-8') as f:
            for entry in jsonl_entries:
                f.write(json.dumps(entry, ensure_ascii=False) + "\n")
        print(f"Archivo exportado correctamente: {output_file}")
        files.download(output_file)
    except Exception as e:
        print(f"Error al exportar el archivo .jsonl: {e}")
        raise
else:
    print("Error: No se encontraron datos válidos para exportar")

# --- Paso 7: Limpieza ---
try:
    shutil.rmtree("azure_indexer_artifacts")  # Limpiar el directorio raíz original
    os.remove(zip_path)
    print("Archivos temporales eliminados correctamente")
except Exception as e:
    print(f"Advertencia: Error al limpiar archivos temporales: {e}")

Sube el archivo ZIP exportado desde Azure Video Indexer


Saving live.zip to live (1).zip
Archivos extraídos correctamente en: azure_indexer_artifacts

Estructura del directorio extraído:
 - azure_indexer_artifacts/__MACOSX/._live
 - azure_indexer_artifacts/__MACOSX/live/._detectedobjects.json
 - azure_indexer_artifacts/__MACOSX/live/._labels.computervision.json
 - azure_indexer_artifacts/__MACOSX/live/._transcript.speechservices.json
 - azure_indexer_artifacts/__MACOSX/live/._Textual_Content_Moderation.json
 - azure_indexer_artifacts/__MACOSX/live/._ocr.json
 - azure_indexer_artifacts/__MACOSX/live/.__KeyFrameThumbnail.zip
 - azure_indexer_artifacts/__MACOSX/live/._emotions.json
 - azure_indexer_artifacts/__MACOSX/live/._contentmoderation.json
 - azure_indexer_artifacts/live/emotions.json
 - azure_indexer_artifacts/live/Textual_Content_Moderation.json
 - azure_indexer_artifacts/live/_KeyFrameThumbnail.zip
 - azure_indexer_artifacts/live/transcript.speechservices.json
 - azure_indexer_artifacts/live/contentmoderation.json
 - azure_indexer_art

AttributeError: 'list' object has no attribute 'download'

In [6]:
# Azure Video Indexer to .jsonl converter
# Compatible con Google Colab

import zipfile
import os
import json
import shutil
import re
try:
    from google.colab import files as colab_files
except ImportError:
    colab_files = None
    print("Advertencia: No se detectó el entorno de Google Colab. La descarga automática no estará disponible.")
from datetime import datetime

# --- Paso 1: Subir archivo ZIP ---
print("Sube el archivo ZIP exportado desde Azure Video Indexer")
try:
    uploaded = colab_files.upload() if colab_files else input("Sube el archivo ZIP manualmente y proporciona la ruta: ")
    zip_path = next(iter(uploaded)) if colab_files else uploaded.strip()
except Exception as e:
    print(f"Error al subir el archivo ZIP: {e}")
    raise

extract_dir = "azure_indexer_artifacts"

# --- Paso 2: Extraer contenido del ZIP ---
try:
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)
    print(f"Archivos extraídos correctamente en: {extract_dir}")
except zipfile.BadZipFile:
    print("Error: El archivo ZIP está dañado o no es válido")
    raise
except Exception as e:
    print(f"Error al extraer el archivo ZIP: {e}")
    raise

# --- Depuración: Mostrar estructura del directorio extraído ---
print("\nEstructura del directorio extraído:")
for root, dirs, files in os.walk(extract_dir):
    for file in files:
        print(f" - {os.path.join(root, file)}")

# --- Paso 3: Pedir metadatos generales ---
print("\nIngresa información contextual para incluir como metadatos comunes")
clase = input("Nombre de la clase o sesión: ").strip()
fecha = input("Fecha (YYYY-MM-DD): ").strip()
curso = input("Curso o unidad (opcional): ").strip()
notas = input("Notas adicionales (opcional): ").strip()

# Validar formato de fecha
try:
    datetime.strptime(fecha, "%Y-%m-%d")
except ValueError:
    print("Error: La fecha debe estar en formato YYYY-MM-DD")
    raise

# --- Paso 4: Funciones para procesar artefactos ---
def extract_transcript(path):
    try:
        with open(path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        entries = []
        for phrase in data.get("RecognizedPhrases", []):
            nbest = phrase.get("NBest", [])
            if nbest:
                text = nbest[0].get("Display", "")
                if text:
                    entries.append({
                        "text": text,
                        "source": "transcript",
                        "metadata": {
                            "clase": clase,
                            "fecha": fecha,
                            "curso": curso,
                            "notas": notas,
                            "tipo": "transcript"
                        }
                    })
        return entries
    except Exception as e:
        print(f"Error al procesar el archivo de transcripción: {e}")
        return []

def extract_ocr(path):
    try:
        with open(path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        entries = []
        for result in data.get("Results", []):
            text = result.get("Ocr", {}).get("content", "")
            if text:
                entries.append({
                    "text": text,
                    "source": "ocr",
                    "metadata": {
                        "clase": clase,
                        "fecha": fecha,
                        "curso": curso,
                        "notas": notas,
                        "tipo": "ocr"
                    }
                })
        return entries
    except Exception as e:
        print(f"Error al procesar el archivo OCR: {e}")
        return []

def extract_labels(path):
    try:
        with open(path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        entries = []
        for result in data.get("Results", []):
            for label in result.get("Label", []):
                name = label.get("name", "")
                if name:
                    entries.append({
                        "text": name,
                        "source": "visual_label",
                        "metadata": {
                            "clase": clase,
                            "fecha": fecha,
                            "curso": curso,
                            "notas": notas,
                            "tipo": "label"
                        }
                    })
        return entries
    except Exception as e:
        print(f"Error al procesar el archivo de etiquetas: {e}")
        return []

def extract_emotions(path):
    try:
        with open(path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        entries = []
        for emotion in data:
            text = emotion.get("Text", "")
            dominant_emotion = emotion.get("DominantEmotion", {}).get("Type", "")
            if text and dominant_emotion:
                entries.append({
                    "text": f"{text} [Dominant Emotion: {dominant_emotion}]",
                    "source": "emotion",
                    "metadata": {
                        "clase": clase,
                        "fecha": fecha,
                        "curso": curso,
                        "notas": notas,
                        "tipo": "emotion",
                        "dominant_emotion": dominant_emotion
                    }
                })
        return entries
    except Exception as e:
        print(f"Error al procesar el archivo de emociones: {e}")
        return []

def extract_detected_objects(path):
    try:
        with open(path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        entries = []
        for result in data.get("results", []):
            obj_type = result.get("type", "")
            for instance in result.get("instances", []):
                if instance.get("confidence", 0) > 0:
                    entries.append({
                        "text": obj_type,
                        "source": "detected_object",
                        "metadata": {
                            "clase": clase,
                            "fecha": fecha,
                            "curso": curso,
                            "notas": notas,
                            "tipo": "detected_object",
                            "confidence": instance.get("confidence", 0)
                        }
                    })
        return entries
    except Exception as e:
        print(f"Error al procesar el archivo de objetos detectados: {e}")
        return []

# --- Paso 5: Buscar y procesar los archivos clave ---
jsonl_entries = []

# Buscar archivos recursivamente
file_paths = {}
for root, _, files in os.walk(extract_dir):
    for file in files:
        file_paths[file] = os.path.join(root, file)

# Procesar archivos si se encuentran
if "transcript.speechservices.json" in file_paths:
    print(f"Procesando transcripción: {file_paths['transcript.speechservices.json']}")
    jsonl_entries.extend(extract_transcript(file_paths["transcript.speechservices.json"]))
else:
    print("Advertencia: No se encontró el archivo de transcripción (transcript.speechservices.json)")

if "ocr.json" in file_paths:
    print(f"Procesando OCR: {file_paths['ocr.json']}")
    jsonl_entries.extend(extract_ocr(file_paths["ocr.json"]))
else:
    print("Advertencia: No se encontró el archivo OCR (ocr.json)")

if "labels.computervision.json" in file_paths:
    print(f"Procesando etiquetas: {file_paths['labels.computervision.json']}")
    jsonl_entries.extend(extract_labels(file_paths["labels.computervision.json"]))
else:
    print("Advertencia: No se encontró el archivo de etiquetas (labels.computervision.json)")

if "emotions.json" in file_paths:
    print(f"Procesando emociones: {file_paths['emotions.json']}")
    jsonl_entries.extend(extract_emotions(file_paths["emotions.json"]))
else:
    print("Advertencia: No se encontró el archivo de emociones (emotions.json)")

if "detectedobjects.json" in file_paths:
    print(f"Procesando objetos detectados: {file_paths['detectedobjects.json']}")
    jsonl_entries.extend(extract_detected_objects(file_paths["detectedobjects.json"]))
else:
    print("Advertencia: No se encontró el archivo de objetos detectados (detectedobjects.json)")

# --- Paso 6: Exportar .jsonl ---
if jsonl_entries:
    sanitized_clase = re.sub(r'[/:*?"<>|]', '_', clase)
    sanitized_clase = sanitized_clase.replace(' ', '_')[:100]  # Truncar a 100 chars
    output_file = f"video_indexed_{sanitized_clase}.jsonl"
    try:
        with open(output_file, "w", encoding='utf-8') as f:
            for entry in jsonl_entries:
                f.write(json.dumps(entry, ensure_ascii=False) + "\n")
        print(f"Archivo exportado correctamente: {output_file}")
        if colab_files:
            try:
                colab_files.download(output_file)
            except Exception as e:
                print(f"Error al descargar el archivo .jsonl: {e}")
                print(f"Por favor, descarga manualmente el archivo desde la carpeta de archivos de Colab: {output_file}")
        else:
            print(f"Descarga manualmente el archivo desde la carpeta de archivos: {output_file}")
    except Exception as e:
        print(f"Error al exportar el archivo .jsonl: {e}")
        raise
else:
    print("Error: No se encontraron datos válidos para exportar")

# --- Paso 7: Limpieza ---
try:
    shutil.rmtree("azure_indexer_artifacts")  # Limpiar el directorio raíz original
    os.remove(zip_path)
    print("Archivos temporales eliminados correctamente")
except Exception as e:
    print(f"Advertencia: Error al limpiar archivos temporales: {e}")

Sube el archivo ZIP exportado desde Azure Video Indexer


Saving live.zip to live (2).zip
Archivos extraídos correctamente en: azure_indexer_artifacts

Estructura del directorio extraído:
 - azure_indexer_artifacts/__MACOSX/._live
 - azure_indexer_artifacts/__MACOSX/live/._detectedobjects.json
 - azure_indexer_artifacts/__MACOSX/live/._labels.computervision.json
 - azure_indexer_artifacts/__MACOSX/live/._transcript.speechservices.json
 - azure_indexer_artifacts/__MACOSX/live/._Textual_Content_Moderation.json
 - azure_indexer_artifacts/__MACOSX/live/._ocr.json
 - azure_indexer_artifacts/__MACOSX/live/.__KeyFrameThumbnail.zip
 - azure_indexer_artifacts/__MACOSX/live/._emotions.json
 - azure_indexer_artifacts/__MACOSX/live/._contentmoderation.json
 - azure_indexer_artifacts/live/emotions.json
 - azure_indexer_artifacts/live/Textual_Content_Moderation.json
 - azure_indexer_artifacts/live/_KeyFrameThumbnail.zip
 - azure_indexer_artifacts/live/transcript.speechservices.json
 - azure_indexer_artifacts/live/contentmoderation.json
 - azure_indexer_art

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Archivos temporales eliminados correctamente
